In this jupyter notebook, we will learn how to download tweets from twitter into a txt file. The goal is to write a function that takes a userid and then saves all of that user's tweets in a text file. We will then, in a different notebook, use this data for natural language processing.

In [ ]:
from twython import Twython, TwythonError
import io
import nltk # nltk provides functions to remove stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re # for regular expression to remove emoji
import validators as vd # to remove urls
from stemming.porter2 import stem # for stemming

In [ ]:
APP_KEY = 'fill_in_your_own'
APP_SECRET = 'fill_in_your_own'
OAUTH_TOKEN = 'fill_in_your_own'
OAUTH_TOKEN_SECRET = 'fill_in_your_own'
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [ ]:
USERNAME = 'swish41'

We now download the timeline of the user and then save all tweets in text file. For this, we have to remove all emojis; we dont want them anyway and they cause troubles when saving

In [ ]:
user_timeline = list()


try:
    user_timeline.extend(twitter.get_user_timeline(screen_name=USERNAME, count = 200))
except TwythonError as e:
    print(e)

number_of_calls = 1
length = len(user_timeline)
current_max = user_timeline[length-1]["id"]    
current_batch = twitter.get_user_timeline(screen_name=USERNAME, max_id = current_max, count = 200)
while current_batch != [] and number_of_calls < 900:
    print(number_of_calls)
    user_timeline.extend(current_batch)
    length = len(current_batch)
    current_max = current_batch[length-1]["id"]
    try:
        current_batch = twitter.get_user_timeline(screen_name=USERNAME, max_id = current_max, count = 200)
    except TwythonError as e:
        print(e)
    number_of_calls += 1



In [ ]:
print(len(user_timeline)) 

In [ ]:
   
def remove_emoji(line):
    NON_BMP_RE = re.compile(u"[^\U00000000-\U0000d7ff\U0000e000-\U0000ffff]", flags=re.UNICODE)
    return NON_BMP_RE.sub(u'', line)
    
    
def save_tweets(timeline, filename):
    file = open(filename, 'w')
    number_of_tweets = len(timeline)
    for i in range(0, number_of_tweets -1):
        tweet = remove_emoji(timeline[i]["text"])
        file.write(tweet)
        file.write(" ")
    file.close()
    
    
save_tweets(user_timeline, 'tweets.txt')
    

Preprocessing: We will now implement stop word elimination as well as link elimination and stemming

In [ ]:
def preprocessing_line(line):
    stop_words = set(stopwords.words('english'))
    #line = remove_emoji(line)
    words = line.split()
    output = ""
    for r in words:
        if (not r in stop_words) and  (not vd.url(r)):
            output += " "
            output += stem(r)
    output += "."
    return output
            
            
def preprocessing(filename):
    file = open(filename)
    output = open('filtered.txt','w')
    for line in file:
        output.write(preprocessing_line(line))
    output.close()
    file.close()
    
        

In [ ]:
preprocessing('tweets.txt')

In [ ]:
len(user_timeline)